In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import sys
import json
from pandas.errors import EmptyDataError
# sys.path.append(os.getenv('Automations'))
import postgresql
pd.set_option('display.max_columns', None)

In [2]:
def combine_data(directory=None, file_paths=[], file_extension='.csv'):
    """
    Combines files in a directory and/or in file_paths.
    Inserts `marketplace` and `date`.
    Returns pandas dataframe.
    """
    # gets all similar file types in a directory
    if directory:
        for dirpath, dirnames, filenames in os.walk(directory):
            for filename in filenames:
                if file_extension in filename:
                    file_paths.append(os.path.join(dirpath, filename))
    
    # combines data
    combined_data = pd.DataFrame()

    for file_path in file_paths:
        print(f"Combining: {file_path}")
        try:
            data = pd.read_csv(file_path, skiprows=0)
        except EmptyDataError as error:
            print(error)
            continue 
            
        if data.empty:
            print("\tEmpty data")
            continue

        if 'Search Terms' in file_path:
            data['date'] = pd.to_datetime(data['date'])
            combined_data = pd.concat([data, combined_data], ignore_index=True)
            cols = ['date'] + [col for col in combined_data.columns if col != 'date']
            combined_data = combined_data[cols]

        else:
            date = re.search(r's - (.*).csv', file_path)[1]
            
            data['Date'] = dt.datetime.strptime(date, '%B %d, %Y').date()
    
            combined_data = pd.concat([data, combined_data], ignore_index=True)
    
            cols = ['Date'] + [col for col in combined_data.columns if col != 'Date']
            combined_data = combined_data[cols]
    
    return combined_data

## Search Term Report

In [3]:
si_search_terms_directory = os.path.join(os.getcwd(), 'PPC Data', 'Scale Insights Downloads', 'Search Terms')
si_search_terms = combine_data(si_search_terms_directory)
si_search_terms.shape

Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/PPC Data/Scale Insights Downloads/Search Terms/Search Terms 2022-10-12.csv
Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/PPC Data/Scale Insights Downloads/Search Terms/Search Terms 2022-10-06.csv
Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/PPC Data/Scale Insights Downloads/Search Terms/Search Terms 2023-01-08.csv
Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/PPC Data/Scale Insights Downloads/Search Terms/Search Terms 2023-06-01.csv
Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-ca

(47736, 36)

In [4]:
si_search_terms.head(3)

,date,Campaign,Keyword,State,Match,Clicks,Orders,Sales,Conversion,Cost,Current Bid,CPC,ACOS,ROAS,Last Activity,Sponsored,Search Terms,Total Impressions,Total Clicks,Total Orders,Total Units,Total Spend,Total Sales,Total CTR,Total CPC,Total Cost /Order,Total Conversion,Total ACOS,Total ROAS,Total P&L,Total BreakEven Bid,Actions,Impressions,marketplace,AdGroup,Units
0,2022-10-23,[OLD] BTS | SBVk-sEX | 'mixology bartender kit...,mixology bartender kit,paused,exact,5,0,0.0,0.0,13.35,0.55,2.67,NaN,NaN,23-Oct-22,Brands,mixology bartender kit,385,5,0,NaN,13.35,0.0,0.0130,2.67,NaN,0.0,NaN,NaN,NaN,NaN,NaN,385.0,US,NaN,NaN
1,2022-10-23,[OLD] BTS | SBVk-sEX | 'bar tools' sv_10k+ | M...,bar tools,paused,exact,1,0,0.0,0.0,2.36,0.55,2.36,NaN,NaN,23-Oct-22,Brands,bar tools,237,3,0,NaN,6.86,0.0,0.0127,2.29,NaN,0.0,NaN,NaN,NaN,NaN,NaN,82.0,US,NaN,NaN
2,2022-10-23,BTS | SBVk-sEX | 'bar tools' sv_10k+ | M2-BS-i...,bar tools,enabled,exact,2,0,0.0,0.0,4.50,0.84,2.25,NaN,NaN,23-Oct-22,Brands,bar tools,237,3,0,NaN,6.86,0.0,0.0127,2.29,NaN,0.0,NaN,NaN,NaN,NaN,NaN,155.0,US,NaN,NaN


### Cleaning

In [5]:
si_sponsored_search_term = si_search_terms[si_search_terms['Sponsored'] == 'Products'].copy()
si_sponsored_search_term.rename(columns={'Campaign': 'campaign_name', 'Keyword': 'targeting', 'State': 'ad_keyword_status', 
                                      'Match': 'match_type', 'Clicks': 'clicks', 'Orders': 'purchases_7d', 'Sales': 'sales_7d', 
                                      'Units': 'units_sold_clicks_7d', 'Cost': 'cost', 'Current Bid': 'keyword_bid',
                                      'CPC': 'cost_per_click', 'ACOS': 'acos_clicks_7d', 'ROAS': 'roas_clicks_7d',
                                      'Search Terms': 'search_term', 'AdGroup': 'ad_group_name', 'Units': 'units_sold_clicks_7d', 
                                      'Impressions': 'impressions'}, inplace=True)
si_sponsored_search_term['campaign_budget_currency_code'] = si_sponsored_search_term['marketplace'] + 'D'
si_sponsored_search_term['click_through_rate'] = si_sponsored_search_term['clicks'] / si_sponsored_search_term['impressions']
si_sponsored_search_term.shape

(29000, 38)

### Search Term Report - Console

In [6]:
search_term_report = pd.read_excel(os.path.join('PPC Data', 'T14.1 Amazon PPC Detailed Reporting (SP) - CLEANED.xlsx'), sheet_name='SP Terms')
search_term_report.shape

(21707, 23)

In [7]:
search_term_report.rename(columns={'currency': 'campaign_budget_currency_code'}, inplace=True)

#### Unneccessary columns

In [8]:
total_columns = [col for col in si_sponsored_search_term.columns if 'Total' in col]
si_sponsored_search_term.drop(columns=['Conversion', 'Last Activity', 'Sponsored', 'Actions'] + total_columns, inplace=True)
search_term_report.drop(columns=['7 Day Conversion Rate'], inplace=True)

#### Adding portfolio name to si data

In [9]:
bulk_sp = pd.read_excel(os.path.join('PPC Data', 'bulk operations.xlsx'), sheet_name='Sponsored Products Campaigns')

sp_cols = {'Portfolio Name (Informational only)': 'portfolio_name', 'Campaign Name (Informational only)': 'campaign_name', 
                       'Ad Group Name (Informational only)': 'ad_group_name', 'Keyword Text': 'keyword', 
                       'Portfolio ID': 'portfolio_id', 'Campaign ID': 'campaign_id', 'Ad Group ID': 'ad_group_id',
                       'Keyword ID': 'keyword_id', 'Match Type': 'match_type'}

bulk_sp = bulk_sp[[col for col in sp_cols.keys()]].drop_duplicates().reset_index(drop=True)
bulk_sp.rename(columns=sp_cols, inplace=True)

bulk_sp

,portfolio_name,campaign_name,ad_group_name,keyword,portfolio_id,campaign_id,ad_group_id,keyword_id,match_type
0,ORIG-BS,BKS | SPk-sEX | 'mixology bartender kit' sv_10...,NaN,NaN,1.927317e+12,112526673940772,NaN,NaN,NaN
1,ORIG-BS,BKS | SPk-sEX | 'mixology bartender kit' sv_10...,NaN,NaN,NaN,112526673940772,NaN,NaN,NaN
2,ORIG-BS,BKS | SPk-sEX | 'mixology bartender kit' sv_10...,BKS | SPk-sEX | 'mixology bartender kit' sv_10...,NaN,NaN,112526673940772,6.256395e+12,NaN,NaN
3,ORIG-BS,BKS | SPk-sEX | 'mixology bartender kit' sv_10...,BKS | SPk-sEX | 'mixology bartender kit' sv_10...,mixology bartender kit,NaN,112526673940772,6.256395e+12,2.780663e+13,Exact
4,ORIG-CB,AKK | SP-AU-CM | - - | - | 2022-10-13 [],NaN,NaN,8.265305e+13,146692843175588,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
38354,ORIG-CB-L1,[OLD] AKS-L1 | B0B2B9WR7X | AU | Bulk Auto 1.3...,[OLD] AKS-L1 | B0B2B9WR7X | Bulk Auto 1.30 | A...,cocktail shaker set,NaN,56349954011308,1.558444e+14,1.750210e+14,Negative Exact
38355,ORIG-CB-L1,[OLD] AKS-L1 | B0B2B9WR7X | AU | Bulk Auto 1.3...,[OLD] AKS-L1 | B0B2B9WR7X | Bulk Auto 1.30 | A...,cocktail shaker,NaN,56349954011308,1.558444e+14,1.285321e+14,Negative Exact
38356,ORIG-CB-L1,[OLD] AKS-L1 | B0B2B9WR7X | AU | Bulk Auto 1.3...,[OLD] AKS-L1 | B0B2B9WR7X | Bulk Auto 1.30 | A...,bartender kit,NaN,56349954011308,1.558444e+14,1.617030e+14,Negative Exact
38357,ORIG-CB-L1,[OLD] AKS-L1 | B0B2B9WR7X | AU | Bulk Auto 1.3...,[OLD] AKS-L1 | B0B2B9WR7X | Bulk Auto 1.30 | A...,wood stand,NaN,56349954011308,1.558444e+14,2.171316e+14,Negative Exact


In [10]:
# Adding portfolio name to SI data
si_sponsored_search_term = si_sponsored_search_term.merge(bulk_sp[['ad_group_name', 'portfolio_name']].drop_duplicates('ad_group_name'), 
                                                          on='ad_group_name', how='left')
# si_sponsored_search_term = si_sponsored_search_term.drop_duplicates().reset_index(drop=True)
si_sponsored_search_term.shape

(29000, 21)

#### Merging search term report and SI data

In [11]:
# Adding keyword bid
search_term_report = search_term_report.merge(si_sponsored_search_term[['date', 'ad_group_name', 'targeting', 'keyword_bid']].drop_duplicates(), 
                         on=['date', 'ad_group_name', 'targeting'], how='left')
search_term_report.shape

(21707, 23)

In [12]:
si_sponsored_search_term_april_may = si_sponsored_search_term[(si_sponsored_search_term.date >= dt.datetime(2023,4,1)) & (si_sponsored_search_term.date <= dt.datetime(2023,5,31))]
si_sponsored_search_term_sept_oct  = si_sponsored_search_term[(si_sponsored_search_term.date >= dt.datetime(2022,9,15)) & (si_sponsored_search_term.date <= dt.datetime(2022,10,7))]
merged_search_term = pd.concat([search_term_report, si_sponsored_search_term_april_may], ignore_index=True)
merged_search_term = pd.concat([search_term_report, si_sponsored_search_term_sept_oct], ignore_index=True)
merged_search_term.shape

(22433, 25)

#### Adding portfolio, campaign, adgroup ids

In [13]:
def remove_parenthesis(value):
    if isinstance(value, str):
        value = re.sub(r'\[.*\]', '', value).strip()
        value = value.replace('{ARCHIVE}', '').strip()
        match = re.search(r"^(?:[^|]*\|){3}", value)
        if match and '| AU |' not in value:
            return match.group(0).strip()
        else:
            return value
            
    else:
        return value

In [14]:
columns = ['ad_group_name', 'campaign_name']
bulk_sp[columns] = bulk_sp[columns].map(remove_parenthesis)

merged_search_term[['cleaned_ad_group_name', 'cleaned_campaign_name']] = merged_search_term[columns].map(remove_parenthesis)
bulk_sp[['cleaned_ad_group_name', 'cleaned_campaign_name']] = bulk_sp[columns].map(remove_parenthesis)

In [15]:
portfolio_id_mapping = bulk_sp.loc[bulk_sp.portfolio_id.notnull(), ['portfolio_name', 'portfolio_id']].copy().drop_duplicates().set_index('portfolio_name')
merged_search_term['portfolio_id'] = merged_search_term.portfolio_name.map(portfolio_id_mapping.portfolio_id)
merged_search_term.shape

(22433, 28)

In [16]:
ad_group_id_mapping = bulk_sp.loc[bulk_sp.ad_group_id.notnull(), ['cleaned_ad_group_name', 'cleaned_campaign_name', 'ad_group_id', 'campaign_id']].copy().drop_duplicates()
merged_search_term = merged_search_term.merge(ad_group_id_mapping, on=['cleaned_campaign_name', 'cleaned_ad_group_name'],
                         how='left')
merged_search_term.drop(columns=['cleaned_ad_group_name', 'cleaned_campaign_name'], inplace=True)
merged_search_term.shape

(22433, 28)

In [17]:
merged_search_term['keyword'] = merged_search_term['targeting']

keyword_id_mapping = bulk_sp.loc[bulk_sp.keyword_id.notnull(), ['keyword', 'match_type', 'keyword_id']].copy().drop_duplicates()

# can't figure out what's keyword id

In [18]:
merged_search_term

,date,portfolio_name,campaign_budget_currency_code,campaign_name,ad_group_name,targeting,match_type,search_term,impressions,clicks,click_through_rate,cost_per_click,cost,sales_7d,acos_clicks_7d,roas_clicks_7d,purchases_7d,units_sold_clicks_7d,units_sold_same_sku_7d,units_sold_other_sku_7d,attributed_sales_same_sku_7d,sales_other_sku_7d,keyword_bid,ad_keyword_status,marketplace,portfolio_id,ad_group_id,campaign_id,keyword
0,2022-07-12,ANJ_PPC,USD,zANJ_CB,ANJ_Manual_Exact,bartender kit,EXACT,bartender kit,1024.0,2,0.001953,$1.00,$2.00,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,1.40,NaN,NaN,1.541365e+13,1.859087e+14,80603878532869,bartender kit
1,2022-07-12,ORIG-CB-L1,USD,"AKS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...","AKS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...",cocktail shaker set,EXACT,cocktail shaker set,6415.0,3,0.000468,$3.00,$9.00,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,3.610068e+13,1.838059e+14,216462420173161,cocktail shaker set
2,2022-07-12,ORIG-CB-L1,USD,"ATS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...","ATS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...",cocktail shaker set,EXACT,cocktail shaker set,24216.0,5,0.000206,$2.40,$12.00,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,3.610068e+13,1.468810e+13,276316246101059,cocktail shaker set
3,2022-07-12,ORIG-BS-L1,USD,"BTS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...","BTS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...",cocktail shaker set,EXACT,cocktail shaker set,4425.0,4,0.000904,$2.35,$9.39,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,1.842730e+14,9.586763e+13,212600336344992,cocktail shaker set
4,2022-07-12,ORIG-BS-L1,USD,"BKS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...","BKS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...",cocktail shaker set,EXACT,cocktail shaker set,4217.0,5,0.001186,$2.00,$10.00,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,1.842730e+14,1.224566e+13,132016407402940,cocktail shaker set
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22428,2022-10-06,ORIG-CB,USD,ATS+AKS | SPk-sEX | 'bar set up' sv_500+ | - |...,ATS+AKS | SPk-sEX | 'bar set up' sv_500+,bar set up,exact,bar set up,26.0,1,0.038462,2.5,2.5,0.0,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,1.13,enabled,US,8.265305e+13,5.507850e+13,160167322041380,bar set up
22429,2022-10-06,ORIG-CB,USD,ATS+AKS | SPk-sEX | 'bar shaker set' sv_2k+ | ...,ATS+AKS | SPk-sEX | 'bar shaker set' sv_2k+,bar shaker set,exact,bar shaker set,76.0,1,0.013158,4.75,4.75,0.0,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,1.35,enabled,US,8.265305e+13,4.309419e+13,236028931443716,bar shaker set
22430,2022-10-06,ORIG-CB,USD,ATS+AKS | SPk-sEX | 'cocktail set bartender ki...,ATS+AKS | SPk-sEX | 'cocktail set bartender ki...,cocktail set bartender kit,exact,cocktail set bartender kit,170.0,1,0.005882,3.41,3.41,0.0,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,1.33,enabled,US,8.265305e+13,1.984942e+14,15360177808556,cocktail set bartender kit
22431,2022-10-06,ORIG-CB,USD,ATS+AKS | SPk-sEX | 'mixed drink shaker kit' s...,ATS+AKS | SPk-sEX | 'mixed drink shaker kit' s...,mixed drink shaker kit,exact,mixed drink shaker kit,8.0,1,0.125000,4.6,4.6,0.0,NaN,NaN,0,0.0,NaN,NaN,NaN,NaN,1.45,enabled,US,8.265305e+13,2.505440e+14,280949361176791,mixed drink shaker kit


#### One last touch

In [19]:
table_name = 'sponsored_products.search_term'
with postgresql.setup_cursor() as cur:
    cur.execute(f"""
        SELECT column_name 
        FROM information_schema.columns
        WHERE table_schema || '.' || table_name = '{table_name}' 
        ORDER BY column_name
    """)
    
    columns = [row['column_name'] for row in cur.fetchall()]
    columns_str = ', '.join(columns)
    
query = f"SELECT {columns_str} FROM {table_name};"
data = postgresql.sql_to_dataframe(query)
data.head()

,acos_clicks_14d,acos_clicks_7d,ad_group_id,ad_group_name,ad_keyword_status,attributed_sales_same_sku_14d,attributed_sales_same_sku_1d,attributed_sales_same_sku_30d,attributed_sales_same_sku_7d,campaign_budget_amount,campaign_budget_currency_code,campaign_budget_type,campaign_id,campaign_name,campaign_status,click_through_rate,clicks,cost,cost_per_click,created_at,date,impressions,keyword,keyword_bid,keyword_id,keyword_type,kindle_edition_normalized_pages_read_14d,kindle_edition_normalized_pages_royalties_14d,marketplace,match_type,portfolio_id,purchases_14d,purchases_1d,purchases_30d,purchases_7d,purchases_same_sku_14d,purchases_same_sku_1d,purchases_same_sku_30d,purchases_same_sku_7d,roas_clicks_14d,roas_clicks_7d,sales_14d,sales_1d,sales_30d,sales_7d,sales_other_sku_7d,search_term,targeting,units_sold_clicks_14d,units_sold_clicks_1d,units_sold_clicks_30d,units_sold_clicks_7d,units_sold_other_sku_7d,units_sold_same_sku_14d,units_sold_same_sku_1d,units_sold_same_sku_30d,units_sold_same_sku_7d,updated_at
0,None,None,191346605902600,AWR | SP-AU-CM | - -,ENABLED,None,None,None,0.0,None,USD,None,161160367347239,AWR | SP-AU-CM | - - | - | 2022-10-13 [],None,0.5,1,1.33,1.33,2023-09-07 14:47:04.966177,2023-06-02,2.0,[Loose Match],1.33,NaN,None,NaN,NaN,US,None,82653048395640.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,None,None,None,None,None,0.0,0.0,bartending set,[Loose Match],NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2023-09-07 15:02:22.104355
1,None,None,191346605902600,AWR | SP-AU-CM | - -,ENABLED,None,None,None,0.0,None,USD,None,161160367347239,AWR | SP-AU-CM | - - | - | 2022-10-13 [],None,0.3333333333333333,1,1.33,1.33,2023-09-07 14:47:04.966177,2023-06-02,3.0,[Loose Match],1.33,NaN,None,NaN,NaN,US,None,82653048395640.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,None,None,None,None,None,0.0,0.0,martini shaker,[Loose Match],NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2023-09-07 15:02:22.104355
2,None,None,191346605902600,AWR | SP-AU-CM | - -,ENABLED,None,None,None,0.0,None,USD,None,161160367347239,AWR | SP-AU-CM | - - | - | 2022-10-13 [],None,0.045454545454545456,1,1.33,1.33,2023-09-07 14:47:04.966177,2023-06-02,22.0,[Loose Match],1.33,NaN,None,NaN,NaN,US,None,82653048395640.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,None,None,None,None,None,0.0,0.0,stainless steel cocktail shaker set,[Loose Match],NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2023-09-07 15:02:22.104355
3,None,None,240398958270925,BWR | SP-AU-SS | - -,ENABLED,None,None,None,0.0,None,USD,None,143980408879328,BWR | SP-AU-SS | - - | - | 2022-10-04 [],None,0.00354924578527063,4,10.68,2.67,2023-09-07 14:47:04.966177,2023-06-02,1127.0,[Close Match],0.61,NaN,None,NaN,NaN,US,None,1927316669387.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,None,None,None,None,None,0.0,0.0,bar set cocktail shaker set,[Close Match],NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2023-09-07 15:02:22.104355
4,None,None,240398958270925,BWR | SP-AU-SS | - -,ENABLED,None,None,None,0.0,None,USD,None,143980408879328,BWR | SP-AU-SS | - - | - | 2022-10-04 [],None,0.0005523844595838704,3,6.66,2.22,2023-09-07 14:47:04.966177,2023-06-02,5431.0,[Close Match],0.61,NaN,None,NaN,NaN,US,None,1927316669387.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,None,None,None,None,None,0.0,0.0,cocktail shaker set,[Close Match],NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2023-09-07 15:02:22.104355


In [20]:
merged_search_term.head()

,date,portfolio_name,campaign_budget_currency_code,campaign_name,ad_group_name,targeting,match_type,search_term,impressions,clicks,click_through_rate,cost_per_click,cost,sales_7d,acos_clicks_7d,roas_clicks_7d,purchases_7d,units_sold_clicks_7d,units_sold_same_sku_7d,units_sold_other_sku_7d,attributed_sales_same_sku_7d,sales_other_sku_7d,keyword_bid,ad_keyword_status,marketplace,portfolio_id,ad_group_id,campaign_id,keyword
0,2022-07-12,ANJ_PPC,USD,zANJ_CB,ANJ_Manual_Exact,bartender kit,EXACT,bartender kit,1024.0,2,0.001953,$1.00,$2.00,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,1.4,NaN,NaN,1.541365e+13,1.859087e+14,80603878532869,bartender kit
1,2022-07-12,ORIG-CB-L1,USD,"AKS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...","AKS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...",cocktail shaker set,EXACT,cocktail shaker set,6415.0,3,0.000468,$3.00,$9.00,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,3.610068e+13,1.838059e+14,216462420173161,cocktail shaker set
2,2022-07-12,ORIG-CB-L1,USD,"ATS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...","ATS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...",cocktail shaker set,EXACT,cocktail shaker set,24216.0,5,0.000206,$2.40,$12.00,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,3.610068e+13,1.468810e+13,276316246101059,cocktail shaker set
3,2022-07-12,ORIG-BS-L1,USD,"BTS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...","BTS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...",cocktail shaker set,EXACT,cocktail shaker set,4425.0,4,0.000904,$2.35,$9.39,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,1.842730e+14,9.586763e+13,212600336344992,cocktail shaker set
4,2022-07-12,ORIG-BS-L1,USD,"BKS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...","BKS-L1 | SP-MN-DD-sKW-EX | ""cocktail shaker se...",cocktail shaker set,EXACT,cocktail shaker set,4217.0,5,0.001186,$2.00,$10.00,$0.00,NaN,0.0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,1.842730e+14,1.224566e+13,132016407402940,cocktail shaker set


In [21]:
# Cleaning targeting
filter = merged_search_term.targeting.str.startswith(r'[')
merged_search_term.loc[filter, 'targeting'] = merged_search_term.loc[filter, 'targeting'].apply(
                                            lambda x: re.match(r'\[(.*)\]', x)[1].lower().replace(' ', '-').strip())

filter = merged_search_term['targeting'].str.startswith('B0')
merged_search_term.loc[filter, 'targeting'] = 'asin="' + merged_search_term.loc[filter, 'targeting'].str.upper() + '"'

merged_search_term.shape

(22433, 29)

In [22]:
merged_search_term['keyword'] = merged_search_term['targeting']

# Match Types
targeting_expression_filter = merged_search_term.targeting.str.startswith('asin')
merged_search_term.loc[targeting_expression_filter, 'match_type'] = 'TARGETING_EXPRESSION'

targeting_expression_predefined_filter = merged_search_term.targeting.isin(['close-match', 'loose-match', 'substitutes', 'complements'])
merged_search_term.loc[targeting_expression_predefined_filter, 'match_type'] = 'TARGETING_EXPRESSION_PREDEFINED'

merged_search_term['keyword_type'] = merged_search_term.match_type

In [23]:
merged_search_term.match_type = merged_search_term.match_type.str.upper()
merged_search_term.ad_keyword_status = merged_search_term.ad_keyword_status.str.upper()
merged_search_term['marketplace'] = 'US'

In [24]:
def replace_dollar(value):
    if pd.isna(value):
        return 0
    if isinstance(value, str):
        return float(value.replace('$', ''))
    else:
        return value

currency_cols = ['cost_per_click', 'cost', 'sales_7d', 'attributed_sales_same_sku_7d', 'sales_other_sku_7d']

for col in currency_cols:
    merged_search_term[col] = merged_search_term[col].apply(replace_dollar)

In [25]:
postgresql.upsert_bulk('sponsored_products.search_term',
                       merged_search_term[(merged_search_term.date >= dt.datetime(2022,9,15)) 
                                           & (merged_search_term.date <= dt.datetime(2022, 10, 7))], 'pandas')

postgresql:    Upserting sponsored_products.search_term
postgresql:    Index(['date', 'portfolio_name', 'campaign_budget_currency_code',
       'campaign_name', 'ad_group_name', 'targeting', 'match_type',
       'search_term', 'impressions', 'clicks', 'click_through_rate',
       'cost_per_click', 'cost', 'sales_7d', 'acos_clicks_7d',
       'roas_clicks_7d', 'purchases_7d', 'units_sold_clicks_7d',
       'units_sold_same_sku_7d', 'units_sold_other_sku_7d',
       'attributed_sales_same_sku_7d', 'sales_other_sku_7d', 'keyword_bid',
       'ad_keyword_status', 'marketplace', 'portfolio_id', 'ad_group_id',
       'campaign_id', 'keyword', 'keyword_type'],
      dtype='object')
postgresql:    date, date: datetime64[ns]
postgresql:    attributed_sales_same_sku_1d, numeric: <class 'float'>
postgresql:    	Missing column: attributed_sales_same_sku_1d
postgresql:    roas_clicks_14d, numeric: <class 'float'>
postgresql:    	Missing column: roas_clicks_14d
postgresql:    units_sold_clicks_1d, i

/Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/postgresql.py:433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].astype(data_type)
/Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/postgresql.py:412: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = np.nan
/

postgresql:    	Upsert Success



In [26]:
merged_search_term[(merged_search_term.date >= dt.datetime(2022,9,15)) 
                                           & (merged_search_term.date <= dt.datetime(2022, 10, 7))]

,date,portfolio_name,campaign_budget_currency_code,campaign_name,ad_group_name,targeting,match_type,search_term,impressions,clicks,click_through_rate,cost_per_click,cost,sales_7d,acos_clicks_7d,roas_clicks_7d,purchases_7d,units_sold_clicks_7d,units_sold_same_sku_7d,units_sold_other_sku_7d,attributed_sales_same_sku_7d,sales_other_sku_7d,keyword_bid,ad_keyword_status,marketplace,portfolio_id,ad_group_id,campaign_id,keyword,keyword_type
21707,2022-09-16,ORIG-BS,USD,BTS+BKS | SPk-sEX | 'cocktail shaker set' sv_2...,BTS+BKS TEST | SPk-sEX | 'cocktail shaker set'...,cocktail shaker set,EXACT,cocktail shaker set,5743.0,5,0.000871,3.57,17.85,50.99,0.3501,2.86,1,1.0,NaN,NaN,0.0,0.0,1.72,ENABLED,US,1.927317e+12,1.759283e+14,238933960487609,cocktail shaker set,exact
21708,2022-09-16,ORIG-BS,USD,BTS+BKS | SPk-sEX | 'bar mixing set' sv_2k+ | ...,BTS+BKS | SPk-sEX | 'bar mixing set' sv_2k+,bar mixing set,EXACT,bar mixing set,29.0,1,0.034483,3.10,3.10,0.00,NaN,NaN,0,0.0,NaN,NaN,0.0,0.0,1.40,ENABLED,US,1.927317e+12,5.751921e+13,114576183144209,bar mixing set,exact
21709,2022-09-16,ORIG-BS,USD,BTS+BKS | SPk-sEX | 'bar kits for bartender' s...,BTS+BKS | SPk-sEX | 'bar kits for bartender' s...,bar kits for bartender,EXACT,bartender kit,63.0,1,0.015873,4.49,4.49,0.00,NaN,NaN,0,0.0,NaN,NaN,0.0,0.0,1.08,ENABLED,US,1.927317e+12,4.496393e+13,144352765037965,bar kits for bartender,exact
21710,2022-09-16,ORIG-BS,USD,BTS+BKS | SPk-sEX | 'bartender set' sv_2k+ | -...,BTS+BKS | SPk-sEX | 'bartender set' sv_2k+,bartender set,EXACT,bartender set,446.0,1,0.002242,2.03,2.03,0.00,NaN,NaN,0,0.0,NaN,NaN,0.0,0.0,1.34,ENABLED,US,1.927317e+12,2.606271e+13,152080718492164,bartender set,exact
21711,2022-09-16,ORIG-BS,USD,BTS+BKS | SPk-sEX | 'cocktail set bartender ki...,BTS+BKS | SPk-sEX | 'cocktail set bartender ki...,cocktail set bartender kit,EXACT,cocktail set bartender kit,70.0,1,0.014286,1.23,1.23,0.00,NaN,NaN,0,0.0,NaN,NaN,0.0,0.0,0.99,ENABLED,US,1.927317e+12,2.652543e+13,156661461127136,cocktail set bartender kit,exact
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22428,2022-10-06,ORIG-CB,USD,ATS+AKS | SPk-sEX | 'bar set up' sv_500+ | - |...,ATS+AKS | SPk-sEX | 'bar set up' sv_500+,bar set up,EXACT,bar set up,26.0,1,0.038462,2.50,2.50,0.00,NaN,NaN,0,0.0,NaN,NaN,0.0,0.0,1.13,ENABLED,US,8.265305e+13,5.507850e+13,160167322041380,bar set up,exact
22429,2022-10-06,ORIG-CB,USD,ATS+AKS | SPk-sEX | 'bar shaker set' sv_2k+ | ...,ATS+AKS | SPk-sEX | 'bar shaker set' sv_2k+,bar shaker set,EXACT,bar shaker set,76.0,1,0.013158,4.75,4.75,0.00,NaN,NaN,0,0.0,NaN,NaN,0.0,0.0,1.35,ENABLED,US,8.265305e+13,4.309419e+13,236028931443716,bar shaker set,exact
22430,2022-10-06,ORIG-CB,USD,ATS+AKS | SPk-sEX | 'cocktail set bartender ki...,ATS+AKS | SPk-sEX | 'cocktail set bartender ki...,cocktail set bartender kit,EXACT,cocktail set bartender kit,170.0,1,0.005882,3.41,3.41,0.00,NaN,NaN,0,0.0,NaN,NaN,0.0,0.0,1.33,ENABLED,US,8.265305e+13,1.984942e+14,15360177808556,cocktail set bartender kit,exact
22431,2022-10-06,ORIG-CB,USD,ATS+AKS | SPk-sEX | 'mixed drink shaker kit' s...,ATS+AKS | SPk-sEX | 'mixed drink shaker kit' s...,mixed drink shaker kit,EXACT,mixed drink shaker kit,8.0,1,0.125000,4.60,4.60,0.00,NaN,NaN,0,0.0,NaN,NaN,0.0,0.0,1.45,ENABLED,US,8.265305e+13,2.505440e+14,280949361176791,mixed drink shaker kit,exact


## Sponsored Brands - Search Term

## Targeting - Sponsored Product

In [234]:
si_keywords_directory = os.path.join(os.getcwd(), 'PPC Data', 'Scale Insights Downloads', 'Keywords')
si_keywords = combine_data(keywords_directory)
si_keywords.head(3)

,Date,ASIN,SKU,Short Name,Type,Campaign,AdGroup,Keyword,Match,State,Impressions,Clicks,Orders,Units,Sales,Conversion %,Spent,Cost Per Click,Clickthrough Rate %,ACOS %,ROAS,TOS Impression Share %,Cost of Goods Sold,FBA Fees,P&L,Current Bid,Break Even Bid
0,2023-01-19,B0B2BJT4G2,BKS,BKS,SP Manual,BTS+BKS | SPk-sEX | 'cocktail shaker set' sv_2...,BTS+BKS TEST | SPk-sEX | 'cocktail shaker set'...,cocktail shaker set,exact,enabled,12468,10,2,2,119.98,20.00,29.91,2.99,0.08,24.93,4.01,NaN,0.0,32.98,57.09,1.01,10.4
1,2023-01-19,NaN,NaN,NaN,SP Manual,ANJ_PT-IP | M&C & ModernM | SlvT-BlkT_08-02,Anj_PT | M&C & ModernM | SlvT_2022-08-02,B07PGVTYS8,NaN,enabled,1498,10,2,2,119.98,20.00,16.87,1.69,0.67,14.06,7.11,NaN,NaN,NaN,103.11,1.00,NaN
2,2023-01-19,NaN,NaN,NaN,SP Manual,ANJ_PT-IP | M&C & ModernM | SlvT-BlkT_08-02,Anj_PT | M&C & ModernM | SlvT_2022-08-02,B07B9KLRDC,NaN,enabled,4730,11,2,2,119.98,18.18,30.97,2.82,0.23,25.81,3.87,NaN,NaN,NaN,89.01,1.75,NaN


In [363]:
sp_keywords = si_keywords[si_keywords.Type.str.startswith('SP')].copy()
sp_keywords.drop(columns=['ASIN', 'SKU', 'Short Name', 'Type', 'Conversion %', 'Cost of Goods Sold', 'FBA Fees', 'P&L', 'Break Even Bid'], inplace=True)
sp_keywords['marketplace'] = 'US'
sp_keywords['campaign_budget_currency_code'] = 'USD'
sp_keywords.rename(columns={'Date': 'date', 'Campaign': 'campaign_name', 'AdGroup': 'ad_group_name', 'Keyword': 'targeting', 
                                      'Match': 'match_type', 'State': 'ad_keyword_status', 'Impressions': 'impressions', 'Clicks': 'clicks', 
                                      'Orders': 'purchases_7d', 'Units': 'units_sold_clicks_7d', 'Sales': 'sales_7d', 'Spent': 'cost', 
                                      'Cost Per Click': 'cost_per_click', 'Clickthrough Rate %': 'click_through_rate', 'ACOS %': 'acos_clicks_7d', 
                                      'ROAS': 'roas_clicks_7d', 'TOS Impression Share %': 'top_of_search_impression_share',
                                      'Current Bid': 'keyword_bid'}, inplace=True)
sp_keywords = sp_keywords.drop_duplicates().reset_index(drop=True)
sp_keywords

,date,campaign_name,ad_group_name,targeting,match_type,ad_keyword_status,impressions,clicks,purchases_7d,units_sold_clicks_7d,sales_7d,cost,cost_per_click,click_through_rate,acos_clicks_7d,roas_clicks_7d,top_of_search_impression_share,keyword_bid,marketplace,campaign_budget_currency_code
0,2023-01-19,BTS+BKS | SPk-sEX | 'cocktail shaker set' sv_2...,BTS+BKS TEST | SPk-sEX | 'cocktail shaker set'...,cocktail shaker set,exact,enabled,12468,10,2,2,119.98,29.91,2.99,0.08,24.93,4.01,NaN,1.01,US,USD
1,2023-01-19,ANJ_PT-IP | M&C & ModernM | SlvT-BlkT_08-02,Anj_PT | M&C & ModernM | SlvT_2022-08-02,B07PGVTYS8,NaN,enabled,1498,10,2,2,119.98,16.87,1.69,0.67,14.06,7.11,NaN,1.00,US,USD
2,2023-01-19,ANJ_PT-IP | M&C & ModernM | SlvT-BlkT_08-02,Anj_PT | M&C & ModernM | SlvT_2022-08-02,B07B9KLRDC,NaN,enabled,4730,11,2,2,119.98,30.97,2.82,0.23,25.81,3.87,NaN,1.75,US,USD
3,2023-01-19,BMG | SPk-sEX | 'gold cocktail shaker' sv_1k+ ...,BMG | SPk-sEX | 'gold cocktail shaker' sv_1k+,gold cocktail shaker,exact,enabled,858,1,1,1,74.99,3.86,3.86,0.12,5.15,19.43,NaN,2.01,US,USD
4,2023-01-19,BTS+BKS | SPk-sEX | 'bar kit' sv_5k+ | - | 202...,BTS+BKS | SPk-sEX | 'bar kit' sv_5k+,bar kit,exact,enabled,270,1,1,1,109.99,1.95,1.95,0.37,1.77,56.41,NaN,2.43,US,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92581,2022-12-18,ATS+AKS | SPp-sEX | TV - B08QYS11XL - CBL Bamb...,ATS+AKS | SPp-sEX | TV - B08QYS11XL - CBL Bamb...,B08QYS11XL,NaN,enabled,2,0,0,0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,1.00,US,USD
92582,2022-12-18,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,B09LCRFP8F,NaN,enabled,1,0,0,0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,1.05,US,USD
92583,2022-12-18,ATS+AKS | SPp-mEX-r3 | GR_sets-without-stands ...,ATS+AKS | SPp-mEX-r3 | GR_sets-without-stands,B08B1TVCRF,NaN,enabled,1,0,0,0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,1.16,US,USD
92584,2022-12-18,BMG | SP-AU-CM | - - | - | 2022-10-04 [],BMG | SP-AU-CM | - -,[Complements],NaN,paused,12,0,0,0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0.80,US,USD


In [364]:
# Cleaning targeting
filter = sp_keywords.targeting.str.startswith(r'[')
sp_keywords.loc[filter, 'targeting'] = sp_keywords.loc[filter, 'targeting'].apply(
                                            lambda x: re.match(r'\[(.*)\]', x)[1].lower().replace(' ', '-').strip())

filter = sp_keywords['targeting'].str.startswith('B0')
sp_keywords.loc[filter, 'targeting'] = 'asin="' + sp_keywords.loc[filter, 'targeting'].str.upper() + '"'

sp_keywords.shape

(92586, 20)

In [365]:
# Adding portfolio name to SI data
sp_keywords = sp_keywords.merge(bulk_sp.loc[bulk_sp.ad_group_name.notnull(), ['ad_group_name', 'portfolio_name']].drop_duplicates('ad_group_name'), 
                                                          on='ad_group_name', how='left')
sp_keywords.shape

(92586, 21)

#### Console Reports

In [366]:
# Combines two reports
targeting_report = pd.read_excel(os.path.join('PPC Data', 'T14.1 Amazon PPC Detailed Reporting (SP) - CLEANED.xlsx'), sheet_name='SP Target')
targeting_report.shape

(113740, 22)

In [367]:
impression_share_report = pd.read_excel(os.path.join('PPC Data', 'T14.1 Amazon PPC Detailed Reporting (SP) - CLEANED.xlsx'), sheet_name='SP ST Impr. Share')
impression_share_report.shape

(12755, 20)

In [368]:
# minor cleaning
filter = impression_share_report['search_term'].str.startswith('b0') & impression_share_report['search_term'].notnull()
impression_share_report.loc[filter, 'search_term'] = 'asin="' + impression_share_report.loc[filter, 'search_term'].str.upper() + '"'

In [369]:
targeting_report = targeting_report.merge(impression_share_report[['date', 'search_term', 'top_of_search_impression_share', 'search_term_impression_rank']],
                left_on=['date', 'targeting'], right_on=['date', 'search_term'], how='left').drop_duplicates().reset_index(drop=True)
targeting_report.drop(columns='search_term', inplace=True)
targeting_report.shape

(113740, 24)

In [370]:
# adding keyword bid
sp_keywords['date'] = pd.to_datetime(sp_keywords.date)
targeting_report = targeting_report.merge(sp_keywords[['date', 'ad_group_name', 'targeting', 'keyword_bid']].drop_duplicates(), on=['date', 'ad_group_name', 'targeting'],
                       how='left')

### Merging SI and console report

In [439]:
dec_filter = (sp_keywords.date > dt.datetime(2022,12,12)) & (sp_keywords.date < dt.datetime(2022,12,26))
mar_filter = (sp_keywords.date > dt.datetime(2023,2,28)) & (sp_keywords.date < dt.datetime(2023,4,2))
filtered_sp_keywords = sp_keywords[dec_filter | mar_filter].copy()
filtered_sp_keywords.shape

(30490, 21)

In [440]:
merged_targeting = pd.concat([targeting_report, filtered_sp_keywords], ignore_index=True)
merged_targeting.reset_index(drop=True, inplace=True)
merged_targeting.sort_values('date')

,date,portfolio_name,campaign_budget_currency_code,campaign_name,ad_group_name,targeting,match_type,impressions,clicks,click_through_rate,cost_per_click,cost,acos_clicks_7d,roas_clicks_7d,sales_7d,purchases_7d,units_sold_clicks_7d,7 Day Conversion Rate,units_sold_same_sku_7d,units_sold_other_sku_7d,attributed_sales_same_sku_7d,sales_other_sku_7d,top_of_search_impression_share,search_term_impression_rank,keyword_bid,ad_keyword_status,marketplace
0,2022-06-27,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,First Ad Group - 4 Launch SKUs - AU $0.50 bid,close-match,-,61,1,0.016393,$0.50,$0.50,NaN,0.00,$0.00,0,0,0.0,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,NaN,NaN
1,2022-06-28,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,First Ad Group - 4 Launch SKUs - AU $0.50 bid,close-match,-,28,0,0.000000,NaN,$0.00,NaN,NaN,$0.00,0,0,NaN,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,NaN,NaN
2,2022-06-28,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,First Ad Group - 4 Launch SKUs - AU $0.50 bid,loose-match,-,4,0,0.000000,NaN,$0.00,NaN,NaN,$0.00,0,0,NaN,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,NaN,NaN
3,2022-06-29,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,First Ad Group - 4 Launch SKUs - AU $0.50 bid,close-match,-,22,0,0.000000,NaN,$0.00,NaN,NaN,$0.00,0,0,NaN,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,NaN,NaN
4,2022-06-29,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,First Ad Group - 4 Launch SKUs - AU $0.50 bid,loose-match,-,0,0,NaN,NaN,$0.00,NaN,NaN,$0.00,0,0,NaN,0.0,0.0,$0.00,$0.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114196,2023-04-01,ORIG-CB,USD,ATS+AKS | SPk-mEX | GR_home - | - | 2022-10-12 [],ATS+AKS | SPk-mEX | GR_home,home bartender kit,exact,25,0,NaN,NaN,0.0,NaN,NaN,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.19,enabled,US
114195,2023-04-01,ORIG-BS,USD,BTS+BKS | SPk-mPH | GR_cocktails2 - | - | 2022...,BTS+BKS | SPk-mPH | GR_cocktails2,cocktail making set,phrase,2,0,NaN,NaN,0.0,NaN,NaN,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.30,enabled,US
114194,2023-04-01,ORIG-BS,USD,BTS+BKS | SPk-mEX | GR_accessories - | - | 202...,BTS+BKS | SPk-mEX | GR_accessories,bar items bar accessories,exact,2,0,NaN,NaN,0.0,NaN,NaN,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.02,enabled,US
114202,2023-04-01,ORIG-CB,USD,ATS+AKS | SPk-mBR | GR_bar-tools - | - | 2022-...,ATS+AKS | SPk-mBR | GR_bar-tools,tool bar,broad,11,0,NaN,NaN,0.0,NaN,NaN,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.13,enabled,US


In [450]:
merged_targeting['match_type'] = merged_targeting.match_type.str.upper()
merged_targeting['ad_keyword_status'] = merged_targeting.ad_keyword_status.str.upper()
merged_targeting['marketplace'] = 'US'
merged_targeting.shape

(144169, 31)

In [442]:
def replace_dollar(value):
    if pd.isna(value):
        return 0
    if isinstance(value, str):
        return float(value.replace('$', ''))
    else:
        return value

currency_cols = ['cost_per_click', 'cost', 'sales_7d', 'attributed_sales_same_sku_7d', 'sales_other_sku_7d']

for col in currency_cols:
    merged_targeting[col] = merged_search_term[col].apply(replace_dollar)

#### Adding portfolio, campaign, adgroup ids

In [443]:
columns = ['ad_group_name', 'campaign_name']

merged_targeting[['cleaned_ad_group_name', 'cleaned_campaign_name']] = merged_targeting[columns].map(remove_parenthesis)
bulk_sp[['cleaned_ad_group_name', 'cleaned_campaign_name']] = bulk_sp[columns].map(remove_parenthesis)

In [444]:
portfolio_id_mapping = bulk_sp.loc[bulk_sp.portfolio_id.notnull(), ['portfolio_name', 'portfolio_id']].copy().drop_duplicates().set_index('portfolio_name')
merged_targeting['portfolio_id'] = merged_targeting.portfolio_name.map(portfolio_id_mapping.portfolio_id)
merged_targeting.shape

(144230, 30)

In [445]:
ad_group_id_mapping = bulk_sp.loc[bulk_sp.ad_group_id.notnull(), ['cleaned_ad_group_name', 'cleaned_campaign_name', 'ad_group_id', 'campaign_id']].copy().drop_duplicates()
merged_targeting = merged_targeting.merge(ad_group_id_mapping, on=['cleaned_campaign_name', 'cleaned_ad_group_name'],
                        how='left')
merged_targeting.drop(columns=['cleaned_ad_group_name', 'cleaned_campaign_name'], inplace=True)

merged_targeting.shape

(144230, 30)

In [446]:
merged_targeting.ad_group_id.isnull().sum()

61

In [447]:
# dropping 61 rows of archived campaigns with $201 total cost
merged_targeting = merged_targeting[~merged_targeting.ad_group_id.isnull()]
merged_targeting.shape

(144169, 30)

### Double-checking with database format

In [449]:
table_name = 'sponsored_products.targeting'
with postgresql.setup_cursor() as cur:
    cur.execute(f"""
        SELECT column_name 
        FROM information_schema.columns
        WHERE table_schema || '.' || table_name = '{table_name}' 
        ORDER BY column_name
    """)
    
    columns = [row['column_name'] for row in cur.fetchall()]
    columns_str = ', '.join(columns)
    
query = f"SELECT {columns_str} FROM {table_name};"
data = postgresql.sql_to_dataframe(query)
data.head()

,acos_clicks_14d,acos_clicks_7d,ad_group_id,ad_group_name,ad_keyword_status,attributed_sales_same_sku_14d,attributed_sales_same_sku_1d,attributed_sales_same_sku_30d,attributed_sales_same_sku_7d,campaign_budget_amount,campaign_budget_currency_code,campaign_budget_type,campaign_id,campaign_name,campaign_status,click_through_rate,clicks,cost,cost_per_click,created_at,date,impressions,keyword,keyword_bid,keyword_id,keyword_type,kindle_edition_normalized_pages_read_14d,kindle_edition_normalized_pages_royalties_14d,marketplace,match_type,portfolio_id,purchases_14d,purchases_1d,purchases_30d,purchases_7d,purchases_same_sku_14d,purchases_same_sku_1d,purchases_same_sku_30d,purchases_same_sku_7d,roas_clicks_14d,roas_clicks_7d,sales_14d,sales_1d,sales_30d,sales_7d,sales_other_sku_7d,search_term_impression_rank,targeting,top_of_search_impression_share,units_sold_clicks_14d,units_sold_clicks_1d,units_sold_clicks_30d,units_sold_clicks_7d,units_sold_other_sku_7d,units_sold_same_sku_14d,units_sold_same_sku_1d,units_sold_same_sku_30d,units_sold_same_sku_7d,updated_at
0,None,None,7243998199406,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0.0,0.0,0.0,15.0,USD,DAILY_BUDGET,198890332775084,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0,0.0,0.0,2023-07-26 17:56:48.031384,2023-04-09,4,"asin=""B08CTP8ZRK""",1.05,215401468381747,TARGETING_EXPRESSION,0,0,US,TARGETING_EXPRESSION,82653048395640.0,0,0,0,0,0,0,0,0,None,None,0.0,0.0,0.0,0.0,0.0,None,"asin=""B08CTP8ZRK""",33.33,0,0,0,0,0,0,0,0,0,2023-09-07 15:05:15.015421
1,None,None,7243998199406,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0.0,0.0,0.0,15.0,USD,DAILY_BUDGET,198890332775084,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0,0.0,0.0,2023-07-26 17:56:48.031384,2023-04-10,1,"asin=""B08CTP8ZRK""",1.05,215401468381747,TARGETING_EXPRESSION,0,0,US,TARGETING_EXPRESSION,82653048395640.0,0,0,0,0,0,0,0,0,None,None,0.0,0.0,0.0,0.0,0.0,None,"asin=""B08CTP8ZRK""",None,0,0,0,0,0,0,0,0,0,2023-09-07 15:05:15.015421
2,None,None,7243998199406,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0.0,0.0,0.0,15.0,USD,DAILY_BUDGET,198890332775084,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0,0.0,0.0,2023-07-26 17:56:48.031384,2023-04-12,1,"asin=""B08CTP8ZRK""",1.05,215401468381747,TARGETING_EXPRESSION,0,0,US,TARGETING_EXPRESSION,82653048395640.0,0,0,0,0,0,0,0,0,None,None,0.0,0.0,0.0,0.0,0.0,None,"asin=""B08CTP8ZRK""",None,0,0,0,0,0,0,0,0,0,2023-09-07 15:05:15.015421
3,None,None,7243998199406,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0.0,0.0,0.0,15.0,USD,DAILY_BUDGET,198890332775084,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0,0.0,0.0,2023-07-26 17:56:48.031384,2023-04-13,2,"asin=""B08CTP8ZRK""",1.05,215401468381747,TARGETING_EXPRESSION,0,0,US,TARGETING_EXPRESSION,82653048395640.0,0,0,0,0,0,0,0,0,None,None,0.0,0.0,0.0,0.0,0.0,None,"asin=""B08CTP8ZRK""",0.0,0,0,0,0,0,0,0,0,0,2023-09-07 15:05:15.015421
4,None,None,7243998199406,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0.0,0.0,0.0,15.0,USD,DAILY_BUDGET,198890332775084,ATS+AKS | SPp-mEX-r1s2 | High rel. w/ ds<5 CBL...,ENABLED,0.0,0,0.0,0.0,2023-07-26 17:56:48.031384,2023-04-14,1,"asin=""B08CTP8ZRK""",1.05,215401468381747,TARGETING_EXPRESSION,0,0,US,TARGETING_EXPRESSION,82653048395640.0,0,0,0,0,0,0,0,0,None,None,0.0,0.0,0.0,0.0,0.0,None,"asin=""B08CTP8ZRK""",None,0,0,0,0,0,0,0,0,0,2023-09-07 15:05:15.015421


In [460]:
merged_targeting['keyword'] = merged_targeting['targeting']

# Match Types
targeting_expression_filter = merged_targeting.targeting.str.startswith('asin')
merged_targeting.loc[targeting_expression_filter, 'match_type'] = 'TARGETING_EXPRESSION'

targeting_expression_predefined_filter = merged_targeting.targeting.isin(['close-match', 'loose-match', 'substitutes', 'complements'])
merged_targeting.loc[targeting_expression_predefined_filter, 'match_type'] = 'TARGETING_EXPRESSION_PREDEFINED'

merged_targeting['keyword_type'] = merged_targeting.match_type

In [458]:
merged_targeting.loc[merged_targeting.match_type == 'TARGETING_EXPRESSION', 'targeting'].value_counts()

targeting
asin="B08BYJ8KSR"             307
asin="B08PT3T5GY"             294
asin="B07TG8BLWW"             289
asin="B07B9KLRDC"             286
asin="B098YGMLTH"             277
                             ... 
asin-expanded="B08BYJF4WF"      1
asin-expanded="B07WN9LCZS"      1
asin-expanded="B09WF2Y5LG"      1
asin-expanded="B09JVNRNCG"      1
asin-expanded="B08BLVQZGW"      1
Name: count, Length: 489, dtype: int64

### Upserting

In [471]:
postgresql.upsert_bulk(table_name, # 3 duplicates
                       merged_targeting.drop_duplicates(['date', 'ad_group_id', 'targeting', 'match_type']), 
                       'pandas')

postgresql:    Upserting sponsored_products.targeting
postgresql:    Index(['date', 'portfolio_name', 'campaign_budget_currency_code',
       'campaign_name', 'ad_group_name', 'targeting', 'match_type',
       'impressions', 'clicks', 'click_through_rate', 'cost_per_click', 'cost',
       'acos_clicks_7d', 'roas_clicks_7d', 'sales_7d', 'purchases_7d',
       'units_sold_clicks_7d', '"7_day_conversion_rate"',
       'units_sold_same_sku_7d', 'units_sold_other_sku_7d',
       'attributed_sales_same_sku_7d', 'sales_other_sku_7d',
       'top_of_search_impression_share', 'search_term_impression_rank',
       'keyword_bid', 'ad_keyword_status', 'marketplace', 'portfolio_id',
       'ad_group_id', 'campaign_id', 'keyword', 'keyword_type',
       'attributed_sales_same_sku_1d', 'roas_clicks_14d',
       'units_sold_clicks_1d', 'attributed_sales_same_sku_14d',
       'attributed_sales_same_sku_30d',
       'kindle_edition_normalized_pages_royalties_14d',
       'units_sold_same_sku_1d', 'campa

/Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/postgresql.py:423: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].astype(data_type)
/Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/postgresql.py:423: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[col

postgresql:    	Upsert Success



## Campaign Placement

In [66]:
placements_directory = os.path.join(os.getcwd(), 'PPC Data', 'Scale Insights Downloads', 'Placements')
si_placements = combine_data(placements_directory)
si_placements.shape

Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/PPC Data/Scale Insights Downloads/Placements/February 1-28, 2023/CampaignPlacementStats - February 6, 2023.csv
Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/PPC Data/Scale Insights Downloads/Placements/February 1-28, 2023/CampaignPlacementStats - February 23, 2023.csv
Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/PPC Data/Scale Insights Downloads/Placements/February 1-28, 2023/CampaignPlacementStats - February 16, 2023.csv
Combining: /Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/PPC Data/Scale Insights D

(61981, 16)

In [71]:
si_placements = si_placements[si_placements.Placement.notnull()].sort_values(['Date', 'Campaign', 'Placement'])
si_placements

,Date,Type,State,Campaign,Placement,Impressions,Clicks,Orders,Units,Sales,Conversion %,Spent,Cost Per Click,Clickthrough Rate %,ACOS %,ROAS
28379,2022-12-12,Auto,Enabled,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Product Pages,20881,11.0,NaN,NaN,NaN,NaN,16.69,1.52,0.05,NaN,NaN
28101,2022-12-12,Auto,Enabled,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Rest of Search,2616,10.0,2.0,2.0,148.98,20.0,15.29,1.53,0.38,10.26,9.74
28225,2022-12-12,Auto,Enabled,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Top of Search,384,5.0,NaN,NaN,NaN,NaN,8.29,1.66,1.30,NaN,NaN
28560,2022-12-12,Auto,Enabled,AKK | SP-AU-LM | - - | - | 2022-10-13 [],Product Pages,58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28670,2022-12-12,Auto,Enabled,AKK | SP-AU-LM | - - | - | 2022-10-13 [],Rest of Search,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,2023-04-02,Manual,Enabled,BWR | SPp-sEX | H&C - B08VC615W2 - BSN Black S...,Rest of Search,178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1276,2023-04-02,Manual,Enabled,BWR | SPp-sEX | H&C - B08VC615W2 - BSN Black S...,Top of Search,18,1.0,NaN,NaN,NaN,NaN,1.00,1.00,5.56,NaN,NaN
1225,2023-04-02,Manual,Enabled,BWR | SPp-sEX | KGr - B0B98XKQ1N - BSN Acrylic...,Product Pages,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
967,2023-04-02,Manual,Enabled,BWR | SPp-sEX | KGr - B0B98XKQ1N - BSN Acrylic...,Rest of Search,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
si_placements.drop(columns=['Type', 'Conversion %'], inplace=True)

si_placements.rename(columns={
                                'Date': 'date', 'State': 'campaign_status', 'Campaign': 'campaign_name', 
                                  'Placement': 'placement_classification', 'Impressions': 'impressions',
                                  'Clicks': 'clicks', 'Orders': 'purchases_7d', 'Units': 'units_sold_clicks_7d', 
                                  'Sales': 'sales_7d', 'Spent': 'cost', 'Cost Per Click': 'cost_per_click', 
                                  'Clickthrough Rate %': 'click_through_rate', 'ACOS %': 'acos_clicks_7d', 'ROAS': 'roas_clicks_7d',
                                  }, inplace=True)

In [74]:
si_placements.shape

(61981, 14)

In [92]:
placement_report = pd.read_excel(os.path.join('PPC Data', 'T14.1 Amazon PPC Detailed Reporting (SP) - CLEANED.xlsx'), sheet_name='SP Place')
placement_report

,date,portfolio_name,campaign_budget_currency_code,campaign_name,campaign_bidding_strategy,placement_classification,impressions,clicks,cost_per_click,cost,sales_7d,acos_clicks_7d,roas_clicks_7d,purchases_7d,units_sold_clicks_7d
0,2022-06-27,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,Dynamic bids - down only,Product pages on Amazon,61,1,$0.50,$0.50,$0.00,NaN,0.0,0,0
1,2022-06-28,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,Dynamic bids - down only,Product pages on Amazon,27,0,NaN,$0.00,$0.00,NaN,NaN,0,0
2,2022-06-28,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,Dynamic bids - down only,Rest of search on Amazon,5,0,NaN,$0.00,$0.00,NaN,NaN,0,0
3,2022-06-29,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,Dynamic bids - down only,Product pages on Amazon,15,0,NaN,$0.00,$0.00,NaN,NaN,0,0
4,2022-06-29,ORIG-ALL-L1,USD,First Campaign - 4 Launch SKUs - AU $0.50 bid,Dynamic bids - down only,Rest of search on Amazon,9,0,NaN,$0.00,$0.00,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97559,2023-02-28,ORIG-CB,USD,AKK | SPp-sEX | M&C - B07TC3D7LL - CBL Box sta...,Dynamic bids - down only,First page Top of Search on Amazon,2,0,NaN,0,0,NaN,NaN,0,0
97560,2023-02-28,ORIG-CB,USD,AKK | SPp-sEX | KGr - B0B42DKGC4 - CBL Acrylic...,Dynamic bids - down only,Product pages on Amazon,4,0,NaN,0,0,NaN,NaN,0,0
97561,2023-02-28,ORIG-CB,USD,AKK | SPp-sEX | KGr - B0B42DKGC4 - CBL Acrylic...,Dynamic bids - down only,First page Top of Search on Amazon,0,0,NaN,0,0,NaN,NaN,0,0
97562,2023-02-28,ORIG-CB,USD,AKK | SPp-sEX | KSS - B08BYJ2HNT - CBL Top Sel...,Dynamic bids - down only,Product pages on Amazon,3,0,NaN,0,0,NaN,NaN,0,0


#### Combining data

In [93]:
dec_filter = (si_placements.date > dt.date(2022,12,12)) & (si_placements.date < dt.date(2022,12,26))
mar_filter = (si_placements.date > dt.date(2023,2,28)) & (si_placements.date < dt.date(2023,4,2))
filtered_si_placements = si_placements[dec_filter | mar_filter].copy()
filtered_si_placements.shape

(28045, 14)

In [94]:
merged_placement = pd.concat([filtered_si_placements, placement_report], ignore_index=True)
merged_placement

,date,campaign_status,campaign_name,placement_classification,impressions,clicks,purchases_7d,units_sold_clicks_7d,sales_7d,cost,cost_per_click,click_through_rate,acos_clicks_7d,roas_clicks_7d,portfolio_name,campaign_budget_currency_code,campaign_bidding_strategy
0,2022-12-13,Enabled,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Product Pages,29613,11.0,2.0,2.0,139.98,16.61,1.51,0.04,11.87,8.43,NaN,NaN,NaN
1,2022-12-13,Enabled,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Rest of Search,2591,17.0,4.0,4.0,314.96,25.67,1.51,0.66,8.15,12.27,NaN,NaN,NaN
2,2022-12-13,Enabled,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Top of Search,376,8.0,3.0,3.0,262.77,12.43,1.55,2.13,4.73,21.14,NaN,NaN,NaN
3,2022-12-13,Enabled,AKK | SP-AU-LM | - - | - | 2022-10-13 [],Product Pages,129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-13,Enabled,AKK | SP-AU-LM | - - | - | 2022-10-13 [],Rest of Search,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125604,2023-02-28 00:00:00,NaN,AKK | SPp-sEX | M&C - B07TC3D7LL - CBL Box sta...,First page Top of Search on Amazon,2,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,ORIG-CB,USD,Dynamic bids - down only
125605,2023-02-28 00:00:00,NaN,AKK | SPp-sEX | KGr - B0B42DKGC4 - CBL Acrylic...,Product pages on Amazon,4,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,ORIG-CB,USD,Dynamic bids - down only
125606,2023-02-28 00:00:00,NaN,AKK | SPp-sEX | KGr - B0B42DKGC4 - CBL Acrylic...,First page Top of Search on Amazon,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,ORIG-CB,USD,Dynamic bids - down only
125607,2023-02-28 00:00:00,NaN,AKK | SPp-sEX | KSS - B08BYJ2HNT - CBL Top Sel...,Product pages on Amazon,3,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,ORIG-CB,USD,Dynamic bids - down only


### Data cleaning

In [99]:
merged_placement['campaign_status'] = merged_placement.campaign_status.str.upper()
merged_placement['placement_classification'] = merged_placement.placement_classification.replace({
                                            'First page Top of Search on Amazon': 'Top of Search on-Amazon',
                                            'Top of Search': 'Top of Search on-Amazon',
                                            'Product pages on Amazon': 'Detail Page on-Amazon',
                                            'Product Pages': 'Detail Page on-Amazon',
                                            'Rest of search on Amazon': 'Other on-Amazon',
                                            'Rest of Search': 'Other on-Amazon',
                                            'Search on-Amazon': 'Other on-Amazon'
})
merged_placement['click_through_rate'] = merged_placement.clicks / merged_placement.impressions
merged_placement['campaign_budget_currency_code'] = 'USD'
merged_placement['marketplace'] = 'US'

In [115]:
def replace_dollar(value):
    if pd.isna(value):
        return 0
    if isinstance(value, str):
        return float(value.replace('$', '').replace('(', '').replace(')', ''))
    else:
        return value

currency_cols = ['cost_per_click', 'cost', 'sales_7d', 'attributed_sales_same_sku_7d']

for col in currency_cols:
    merged_placement[col] = merged_placement[col].apply(replace_dollar)

In [105]:
merged_placement['cleaned_campaign_name'] = merged_placement.campaign_name.apply(remove_parenthesis)

In [110]:
campaign_id_mapping = bulk_sp.loc[bulk_sp.campaign_id.notnull(), ['cleaned_campaign_name', 'campaign_id']].copy().drop_duplicates()
merged_placement = merged_placement.merge(campaign_id_mapping, on='cleaned_campaign_name', how='left')
merged_placement.drop(columns=['cleaned_campaign_name'], inplace=True)
merged_placement.campaign_id.isnull().sum()

42

In [111]:
merged_placement = merged_placement[merged_placement.campaign_id.notnull()]
merged_placement

,date,campaign_status,campaign_name,placement_classification,impressions,clicks,purchases_7d,units_sold_clicks_7d,sales_7d,cost,cost_per_click,click_through_rate,acos_clicks_7d,roas_clicks_7d,portfolio_name,campaign_budget_currency_code,campaign_bidding_strategy,marketplace,cleaned_campaign_name,campaign_id
0,2022-12-13,ENABLED,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Detail Page on-Amazon,29613,11.0,2.0,2.0,139.98,16.61,1.51,0.000371,11.87,8.43,NaN,USD,NaN,US,AKK | SP-AU-CM | - - |,1.466928e+14
1,2022-12-13,ENABLED,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Other on-Amazon,2591,17.0,4.0,4.0,314.96,25.67,1.51,0.006561,8.15,12.27,NaN,USD,NaN,US,AKK | SP-AU-CM | - - |,1.466928e+14
2,2022-12-13,ENABLED,AKK | SP-AU-CM | - - | - | 2022-10-13 [],Top of Search on-Amazon,376,8.0,3.0,3.0,262.77,12.43,1.55,0.021277,4.73,21.14,NaN,USD,NaN,US,AKK | SP-AU-CM | - - |,1.466928e+14
3,2022-12-13,ENABLED,AKK | SP-AU-LM | - - | - | 2022-10-13 [],Detail Page on-Amazon,129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USD,NaN,US,AKK | SP-AU-LM | - - |,1.961317e+14
4,2022-12-13,ENABLED,AKK | SP-AU-LM | - - | - | 2022-10-13 [],Other on-Amazon,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USD,NaN,US,AKK | SP-AU-LM | - - |,1.961317e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125604,2023-02-28 00:00:00,NaN,AKK | SPp-sEX | M&C - B07TC3D7LL - CBL Box sta...,Top of Search on-Amazon,2,0.0,0.0,0.0,0,0,NaN,0.000000,NaN,NaN,ORIG-CB,USD,Dynamic bids - down only,US,AKK | SPp-sEX | M&C - B07TC3D7LL - CBL Box sta...,2.166084e+13
125605,2023-02-28 00:00:00,NaN,AKK | SPp-sEX | KGr - B0B42DKGC4 - CBL Acrylic...,Detail Page on-Amazon,4,0.0,0.0,0.0,0,0,NaN,0.000000,NaN,NaN,ORIG-CB,USD,Dynamic bids - down only,US,AKK | SPp-sEX | KGr - B0B42DKGC4 - CBL Acrylic...,2.871953e+13
125606,2023-02-28 00:00:00,NaN,AKK | SPp-sEX | KGr - B0B42DKGC4 - CBL Acrylic...,Top of Search on-Amazon,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,ORIG-CB,USD,Dynamic bids - down only,US,AKK | SPp-sEX | KGr - B0B42DKGC4 - CBL Acrylic...,2.871953e+13
125607,2023-02-28 00:00:00,NaN,AKK | SPp-sEX | KSS - B08BYJ2HNT - CBL Top Sel...,Detail Page on-Amazon,3,0.0,0.0,0.0,0,0,NaN,0.000000,NaN,NaN,ORIG-CB,USD,Dynamic bids - down only,US,AKK | SPp-sEX | KSS - B08BYJ2HNT - CBL Top Sel...,9.919531e+13


In [125]:
postgresql.upsert_bulk('sponsored_products.campaign_placement', # 552 rows duplicated values all from console `campaign placement report`
                       merged_placement.drop_duplicates(['date', 'campaign_id', 'placement_classification']), 
                       'pandas')

postgresql:    Upserting sponsored_products.campaign_placement
postgresql:    Index(['date', 'campaign_status', 'campaign_name', 'placement_classification',
       'impressions', 'clicks', 'purchases_7d', 'units_sold_clicks_7d',
       'sales_7d', 'cost', 'cost_per_click', 'click_through_rate',
       'acos_clicks_7d', 'roas_clicks_7d', 'portfolio_name',
       'campaign_budget_currency_code', 'campaign_bidding_strategy',
       'marketplace', 'cleaned_campaign_name', 'campaign_id',
       'attributed_sales_same_sku_1d', 'units_sold_clicks_1d',
       'attributed_sales_same_sku_7d', 'attributed_sales_same_sku_14d',
       'sales_1d', 'campaign_rule_based_budget_amount',
       'attributed_sales_same_sku_30d',
       'kindle_edition_normalized_pages_royalties_14d',
       'purchases_same_sku_14d', 'spend', 'purchases_same_sku_1d',
       'campaign_budget_type', 'units_sold_same_sku_1d', 'purchases_1d',
       'purchases_same_sku_7d', 'units_sold_same_sku_7d',
       'campaign_budget_amo

/Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/postgresql.py:423: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[column_name].astype(data_type)
/Users/calvin/Library/CloudStorage/GoogleDrive-calvin@barebarrel.com/Shared drives/BB: Shared Drive/Calvin - Personal Folder/Bare Barrel Automations/postgresql.py:423: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column_name] = data[col

postgresql:    	Upsert Success

